In [ ]:
# python 3.6 以上とpandasを入れておくこと
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)
import warnings
warnings.filterwarnings('ignore')
import math
from pprint import pprint

In [ ]:
# 定数の定義

ALL_EVENTS = [
    '333',
    '222', 
    '444',
    '555',
    '666',
    '777',
    '333bf',
    '333fm',
    '333oh',
    '333ft',
    'clock',
    'minx',
    'pyram',
    'skewb',
    'sq1',
    '444bf',
    '555bf',
    '333mbf',
]
assert len(ALL_EVENTS) == 18, '全競技の数は18のはずですが、{}でした。'.format(len(ALL_EVENTS))


In [ ]:
# 入力ファイル、パラメータとかのいじる部分

# WCAからダウンロードして、CubeCompsに挿入したのと同じファイルのパス
# ID揃えるために順序変えていない同一ファイルを入れること
input_csv_file = "input.csv" 

# 出力ファイルのパス
output_file_csv = "output.csv"
output_file_tsv = "output.tsv"

# 参加種目よりもスタッフ数が多い人々
output_file_tsv_high_load_people =  "high_load_people.tsv"

# 参加種目よりもスタッフ数が少ない人々
output_file_tsv_low_load_people =  "low_load_people.tsv"


# https://www.worldcubeassociation.org/results/misc/export.html からダウンロードしたランキングファイルのパス
wca_RanksSingle_tsv_file = "WCA_export_RanksSingle.tsv"

# 競技卓の数
num_of_competition_tables = 12

# 今大会で行われる各種目ごとの必要スタッフ人数
# 基本的に、ジャッジは競技卓の数と同じ人数とする
num_staffs = {
    "333": {"S": 3, "J": (num_of_competition_tables + 0) },
    "222": {"S": 3, "J": (num_of_competition_tables + 0) },
    "333oh": {"S": 4, "J": (num_of_competition_tables + 0) },
    "333bf": {"S": 4, "J": (num_of_competition_tables + 0) },
}
# 間違った種目名にしていないか確認
assert all([ (event in ALL_EVENTS) for event in num_staffs.keys() ]), 'num_staffsの種目名に誤りがあります。'

# スタッフの人数が足りなかった場合に処理を続行するかどうか
# Trueの場合、可能な限りの人数をジャッジに割り当てつつ、処理を続行
# Falseの場合、エラーを出力して停止。
# 問題がある割り当てに気付けるように基本的にはFalseにしておいて、問題が起こったら一時的にTrueにしてから手作業で調整
continue_when_judges_run_out = False

# スタッフを割り当てない人をここにリストアップする (運営スタッフ , 子供, 外国人等)
# 初参加の人には自動で割り当てないようにするので、含める必要はない
staff_blacklist = ["2016MANN01", "2017FARN01", "2017FARN02", "2006NISH01", "2017SHIK01", "2016XIAN08", "2018YAMA02", "2012YOSH02", "2013KIMD01", "2018PILE01", "2017HIKI01", "2017OGAT01", "2016GABA02", "2018YOSH03", "2017FURU04", "2017TAKE04", "2017TAKE05", "2014ASHI01", "2018KITA01", "2015NUPU01", "2018KURO04", "2012JIAN06", "2010TANA02", "2018NOJI01", "2005SUSE01", "2017ALIB02", "2010SICH01"] 

In [ ]:
# 申込みデータのロード
df = pd.read_csv(input_csv_file)
# 個人情報がみえるとやばいので不要カラム除去。実際の利用時は除去しなくてよいのでこの行はコメントアウトする。
df = df .drop(["Status", "Birth Date", "Email", "Gender", "Guests", "IP"], axis=1)
df.index += 1 
df.index.name = "CubeComps ID"
df.head()

In [ ]:
print('競技者数')
for event in num_staffs.keys():
    print("{event}: {num}".format(event=event, num=len(df[df[event] == 1])))
print()

# 各種目ごとのグループ数は(競技卓の数の2倍)を基準に決定
# 例: 117人で競技卓12なら、24で割って4あまり21なので5グループとする
num_groups = { event: math.ceil(len(df[df[event] == 1]) / (num_of_competition_tables * 2)) for event in num_staffs.keys() }

print('競技卓の数と参加者数から自動決定したグループ数 (※問題がある場合は手入力で直接編集してください)')
'''
# 例
num_groups = {
  '222': 3,
  '333': 5,
  '333bf': 2,
  '333oh': 2
}
'''
pprint(num_groups, width=3)



In [ ]:
group_labels = ["A", "B", "C", "D", "E", "F"]
events = num_groups.keys()
group_names = {event: [f"{event}_{label}" for label in group_labels[:num_groups[event]]] for event in events}

In [ ]:
# WCAの単発ランキング情報のロード
results_df = pd.read_table(wca_RanksSingle_tsv_file, dtype={'eventId': str})
results_df.head()

In [ ]:
#  グループ分けに使うカラムの追加
df = df.assign(staffable=0).assign(**{f"{event}_rank": 999999 for event in events})
df["event_count"] = df[list(events)].sum(axis=1).astype(int)

# スタッフ割り当て数と競技数は横並びのほうが見やすいので、この場所でカラムを宣言しておく
df = df.assign(staff_count=0)

for wcaid in df["WCA ID"][df["WCA ID"].notnull()]:
    staffable = 0 if wcaid in staff_blacklist else 1
    df.loc[df["WCA ID"] == wcaid, "staffable"] = staffable
    for event in events:
        temp_series = results_df.query(f'personId == "{wcaid}" and eventId == "{event}"')["worldRank"]
        if not temp_series.empty:
            rank = temp_series.item()
            df.loc[df["WCA ID"] == wcaid, f"{event}_rank"] = rank
df.head()

In [ ]:
# 各種目のグループカラムの初期化
df = df.assign(**{group_name: 0 for group_name in sum(group_names.values(), [])})

In [ ]:
# 競技グループ分け
# 速い人から順にA, B, C… と等分するだけだと、あとのグループに経験が足りない人が偏り厳しいので、
# まずスタッフができる人を速い方から(グループ数*2)個の群に等分し、順に A, B, C, A, B, C のように割り当て、次に残りを等分する
for (event, group_columns) in group_names.items():
    n = len(group_columns)
    competitors = df[df[event]==1]
    staffable_competitors = competitors.query("staffable == 1").sort_values(f"{event}_rank").index
    unstaffable_competitors = competitors.query("staffable == 0").sort_values(f"{event}_rank").index
    for (group_column, group) in zip(group_columns * 3, np.array_split(staffable_competitors, n*2) + np.array_split(unstaffable_competitors, n)):
        for idx in group:
            df.loc[idx, group_column] = 1

In [ ]:
# スタッフ割当
for (event, gnames) in group_names.items():
    for (i_group, group_column) in enumerate(gnames):
        staffable_df = df[df["staffable"] == 1][df[group_column]  == 0]
        staffable_df["staff_count-event_count"] = staffable_df["staff_count"] - staffable_df["event_count"]
        # その種目のグループ数が複数グループの場合
        if len(gnames) > 1:
            prev_group_column = gnames[i_group-1]
            staff_candidates = np.r_[
                #  原則、1つ前のグループの人に仕事を割り当てる。優先順序はその種目のランキング順。
                staffable_df[staffable_df[prev_group_column] == 1].sort_values(f"{event}_rank").index,
                # 足りない場合、同じ種目の1つ前以外のグループからも採用する。 
                # 優先順位はここまででのスタッフ割当数の少ない人。割当数が同じ場合は、各種目のランキング順。
                staffable_df[staffable_df[prev_group_column]  == 0][staffable_df[event] == 1].sort_values(["staff_count", "staff_count-event_count"] + [f"{e}_rank" for e in events]).index,
                # 同じ種目の参加者で足りない場合は、その種目に参加しない人からも採用する。優先順位同じ。
                staffable_df[staffable_df[prev_group_column]  == 0][staffable_df[event] == 0].sort_values(["staff_count", "staff_count-event_count"] + [f"{e}_rank" for e in events]).index
            ]
        # その種目のグループ数が1の場合
        else: 
            # その種目に参加しない人から採用する。
            # 優先順位はここまででのスタッフ割当数の少ない人。割当数が同じ場合は、各種目のランキング順。
            staff_candidates = staffable_df[staffable_df[event] == 0].sort_values(["staff_count", "staff_count-event_count"] + [f"{e}_rank" for e in events]).index

        for (role, num_needed) in num_staffs[event].items():
            error_msg = "{}でスタッフ足りなくて割当無理でした: {} < {}".format(group_column, len(staff_candidates), num_needed)
            if len(staff_candidates) < num_needed and (not continue_when_judges_run_out):
                raise Exception(error_msg)
            
            # 後ろの条件はなくても挙動は変わらないが、読みやすいように明示しておいた
            if len(staff_candidates) < num_needed and continue_when_judges_run_out:
                # 可能な人員を最大限割り当てて処理を続行
                print(error_msg)
                num_needed = len(staff_candidates)
            
            staff_ids = staff_candidates[:num_needed]
            staff_candidates = staff_candidates[num_needed:]
            for staff in staff_ids:
                df.loc[staff, group_column] = role
                df.loc[staff, "staff_count"] += 1

In [ ]:
# グループ分け・スタッフ割り当て結果
df

In [ ]:
# 種目数よりスタッフ数が多くて、負担が多い人一覧"
high_load_df = df.query("staff_count > event_count").sort_values("staff_count", ascending=False)[["Name", "WCA ID", "staff_count", "event_count"]]
high_load_df.to_csv(output_file_tsv_high_load_people, sep='\t' )
high_load_df

In [ ]:
# 種目数よりスタッフ数が少なくて、楽な人一覧
low_load_df = df.query("staff_count < event_count and staffable == 1").sort_values("staff_count", ascending=False)[["Name", "WCA ID", "staff_count", "event_count"]]
low_load_df.to_csv(output_file_tsv_low_load_people, sep='\t' )
low_load_df

In [ ]:
# 種目数とスタッフ数が同じ人一覧
df.query("staff_count == event_count").sort_values("staff_count", ascending=False)[["Name", "WCA ID", "staff_count", "event_count"]]

In [ ]:
# 割当数確認
for group_name in sum(group_names.values(), []):
    print(f"{group_name} の役割割当数") 
    print(df[group_name].value_counts())
    print()

In [ ]:
# 最初の種目の最初のラウンドが初参加者なケース (いたら適当に手動でBに移すとかして調整して下さい)
first_group = list(group_names.values())[0][0]
df[df["WCA ID"].isnull()][df[first_group]  == 1][["Name", "WCA ID", first_group]]

In [ ]:
# 結果ファイルの出力
df.to_csv(output_file_csv)
df.to_csv(output_file_tsv, sep='\t' )

# そのままエクセルで開くと文字化けしちゃうので
# ＄ cat output.tsv | pbcopy
# みたいにして、エクエルにコピペすると良いと思う (Macの場合)